In [9]:
import pymysql
import pandas as pd
import numpy as np

# 从数据库中读取情感分析的结果
connect = pymysql.connect(host="localhost", user="root", password="123", port=3306, db="zhihu")
cursor = connect.cursor()
cursor.execute(f"select * from zhihu")
result = cursor.fetchall()
connect.commit()
cursor.close()
connect.close()

In [10]:
# 对情感分析的结果归一化，数据映射到0～1范围之内处理
res = pd.DataFrame(result)
res.drop_duplicates(subset=0,keep='first',inplace=True)
res = res.drop([0], axis=1)
res = res.apply(lambda x: (x - x.min())/(x.max()-x.min()))

male = res[res[2] == 0.0][1]
female = res[res[2] == 1.0][1]

In [11]:
# 将0～1划分为五段，使用pd.cut方法统计每个区间段的个数
split_num = 5
male_cut = pd.cut(male, bins=list(map(lambda x: x/split_num, range(0,split_num+1)))).value_counts().sort_index(ascending = False)
female_cut = pd.cut(female, bins=list(map(lambda x: x/split_num, range(0,split_num+1)))).value_counts().sort_index(ascending = False)
(male_cut, female_cut)

((0.8, 1.0]    3728
 (0.6, 0.8]     743
 (0.4, 0.6]     283
 (0.2, 0.4]     168
 (0.0, 0.2]     115
 Name: 1, dtype: int64,
 (0.8, 1.0]    6834
 (0.6, 0.8]    1116
 (0.4, 0.6]     452
 (0.2, 0.4]     219
 (0.0, 0.2]     188
 Name: 1, dtype: int64)

In [12]:
# 下面使用Pyecharts作图

from pyecharts.charts import Bar, Pie, Page
from pyecharts import options as opts

page = Page()

# 初始化饼图参数
pie_male = Pie().add(
    series_name="负面情绪值",
    # 设置键值对，这里注意pd.cut函数自动生成的(0, 0.2]的格式是pandas._libs.interval.Interval，需要转换为str格式
    data_pair=list(map(lambda x: (str(x[0]), x[1]), list(male_cut.items())))
).set_global_opts(
    # 设置标题内容、字体、颜色
    title_opts=opts.TitleOpts(title="负面情绪值 - 男", title_textstyle_opts=opts.TextStyleOpts(color="#000")),
    # 不显示图例
    legend_opts=opts.LegendOpts(is_show=False),
).set_series_opts(
    # 设置百分比格式
    tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: {c} ({d}%)")
)

# 初始化饼图参数
pie_female = Pie().add(
    series_name="负面情绪值",
    data_pair=list(map(lambda x: (str(x[0]), x[1]), list(female_cut.items())))
).set_global_opts(
    title_opts=opts.TitleOpts(title="负面情绪值 - 女", title_textstyle_opts=opts.TextStyleOpts(color="#000")),
    legend_opts=opts.LegendOpts(is_show=False),
).set_series_opts(
    tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: {c} ({d}%)")
)

page.add(pie_male)
page.add(pie_female)

# 在notebook中显示饼图
page.render_notebook()

c:\users\abc\appdata\local\programs\python\python38\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
c:\users\abc\appdata\local\programs\python\python38\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
